In [1]:
# 导入模块
import os # 用于操作系统的交互，例如获取环境变量。
import openai # OpenAI 的官方库，用于与 OpenAI 服务进行交互。
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY'] # 从环境变量中获取 OPENAI_API_KEY 作为 API 密钥。
openai.api_type = "azure"
openai.api_base = "https://zjx.openai.azure.com/"
openai.api_version = "2023-05-15"

In [2]:
# 定义了一个名为 get_completion 的函数，其主要目的是使用 OpenAI 的 API 为给定的 prompt 获取模型的回复
def get_completion(prompt):
    # messages 是一个列表，其中包含一个字典，这个字典定义了一个用户角色和他们的消息内容。
    # 这是 OpenAI 的 Chat API 接受的格式，它允许多次交互（例如，先由用户发送消息，然后由模型回复，然后再由用户发送消息，等等）
    messages = [{"role": "user", "content": prompt}] 
    # 使用 try 和 except 来尝试执行某些代码，并在出现异常时捕获该异常。
    try:
        response = openai.ChatCompletion.create(
            engine="gpt35",  # 指定使用哪个模型。在这里，它是 "gpt35"。
            messages=messages,
            temperature=0, # 控制输出随机性的参数。值为 0 表示模型会产生最确定性的输出；较高的值（例如 1.0）会产生更多的随机性。
        )
        return response.choices[0].message["content"] # 从模型的响应中获取其消息内容并返回。
    except Exception as e:
        print(f"Error: {e}")
        return None

In [3]:
## 在这里，max_tokens主要限制的是输出的令牌数量，也就是模型生成的内容的最大长度。
def get_completion_from_messages(messages, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        max_tokens=max_tokens,
        engine="gpt35",
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [4]:
# 分类 & 结构化输出

delimiter = "####"
system_message = f"""
您将收到客户服务查询。\
每个客户服务查询都将用 \
{delimiter} 字符进行分隔。
请将每个查询分类为一个主类别 \
和一个次类别。
请以 json 格式提供您的输出，使用 \
的键为：primary 和 secondary。

主类别：账单、技术支持、\
账户管理或一般咨询。

账单次类别：
退订或升级
添加付款方式
费用说明
争议费用

技术支持次类别：
常规故障排除
设备兼容性
软件更新

账户管理次类别：
密码重置
更新个人信息
关闭账户
账户安全

一般咨询次类别：
产品信息
定价
反馈
与人沟通
"""

user_message = f"""\
我希望您删除我的个人资料和所有用户数据"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)


{
    "primary": "账户管理",
    "secondary": "关闭账户"
}
